## screener

In [2]:
import requests
import pandas as pd

from bs4 import BeautifulSoup

In [3]:
# URL of the page to scrape
url = "https://www.screener.in/company/TATAMOTORS/consolidated/"

# Send a GET request to the webpage
response = requests.get(url)

In [ ]:
from bs4 import BeautifulSoup
import pandas as pd
import re

def clean_value(value):
    # Remove extra whitespace and newlines
    value = ' '.join(value.split())
    
    # Handle currency values (₹)
    if '₹' in value:
        value = value.replace('₹', '₹ ')
        # Remove extra spaces between digits and commas
        value = re.sub(r'\s+([0-9,])', r'\1', value)
    
    # Handle percentage values
    if '%' in value:
        # Clean up percentage values
        value = value.replace(' %', '%')
        value = re.sub(r'\s+', '', value)
    
    return value


# Parse the HTML content
soup = BeautifulSoup(html_content, 'html.parser')

# Extract and clean data
data = []
for item in soup.find_all('li', class_='flex'):
    name = item.find('span', class_='name').text.strip()
    value = item.find('span', class_='value').text.strip()
    # Clean the value
    cleaned_value = clean_value(value)
    data.append({'Metric': name, 'Value': cleaned_value})

# Create DataFrame
df = pd.DataFrame(data)

# Set display options for pandas
pd.set_option('display.max_colwidth', None)
pd.set_option('display.width', None)

# Display the table
print("\nCompany Ratios:")
print(df.to_string(index=False))

# Save to CSV with proper formatting
df.to_csv('company_ratios.csv', index=False)
print("\nData has been saved to 'company_ratios.csv'")


Company Ratios:
        Metric         Value
    Market Cap ₹2,89,492 Cr.
 Current Price          ₹786
    High / Low   ₹1,179 /696
     Stock P/E          8.65
    Book Value          ₹275
Dividend Yield         0.38%
          ROCE         20.1%
           ROE         49.4%
    Face Value         ₹2.00

Data has been saved to 'company_ratios.csv'


In [4]:
response

<Response [200]>

In [5]:
# Parse the content with BeautifulSoup
soup = BeautifulSoup(response.content, 'html.parser')

In [6]:
# Find the Pros section
pros_section = soup.find('div', class_='pros')
pros_list = pros_section.find_all('li') if pros_section else []

print("Pros:")
for pro in pros_list:
    print(pro.text.strip())

# Find the Cons section
cons_section = soup.find('div', class_='cons')
cons_list = cons_section.find_all('li') if cons_section else []

print("\nCons:")
for con in cons_list:
    print(con.text.strip())

Pros:
Company has reduced debt.
Company has delivered good profit growth of 93.1% CAGR over last 5 years

Cons:
Stock is trading at 2.86 times its book value
Promoter holding has decreased over last quarter: -3.78%
Tax rate seems low


In [7]:
# Find the table under the 'quarters' section
quarters_section = soup.find('section', id='quarters')
table = quarters_section.find('table')

# Extract the header (quarters)
headers = [th.text.strip() for th in table.find_all('th')]

# Extract the data rows
rows = []
for tr in table.find_all('tr')[1:]:
    cells = [td.text.strip() or f"https://www.screener.in{td.find('a', href=True)['href']}" for td in tr.find_all('td')]

    rows.append(cells)

# Create a DataFrame
quarters_section_df = pd.DataFrame(rows, columns=headers)
quarters_section_df.head()

,,Sep 2021,Dec 2021,Mar 2022,Jun 2022,Sep 2022,Dec 2022,Mar 2023,Jun 2023,Sep 2023,Dec 2023,Mar 2024,Jun 2024,Sep 2024
0,Sales +,"61,379","72,229","78,439","71,935","79,611","88,489","105,932","102,236","105,129","110,577","119,986","108,048","101,450"
1,Expenses +,"57,262","65,151","70,156","69,522","74,039","77,668","92,818","89,019","91,362","95,159","102,851","92,263","89,291"
2,Operating Profit,"4,117","7,078","8,283","2,413","5,572","10,820","13,114","13,217","13,767","15,418","17,135","15,785","12,159"
3,OPM %,7%,10%,11%,3%,7%,12%,12%,13%,13%,14%,14%,15%,12%
4,Other Income +,862,789,189,"2,381","1,351","1,130","1,453",895,"1,557","1,604","1,619","1,747","1,647"


In [8]:
# Find the section with id="profit-loss"
profit_loss_section = soup.find('section', id='profit-loss')

# Extract the table within this section
table = profit_loss_section.find('table', class_='data-table')

# Extract headers
profit_loss_section_headers = [header.text.strip() for header in table.find_all('th')]

# Extract rows
rows = []
for row in table.find_all('tr')[1:]:  # Skip the header row
    cells = [cell.text.strip() for cell in row.find_all('td')]
    rows.append(cells)

# Create a DataFrame
profit_loss_section_df = pd.DataFrame(rows, columns=profit_loss_section_headers)
profit_loss_section_df.head()

,,Mar 2013,Mar 2014,Mar 2015,Mar 2016,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Mar 2024,TTM
0,Sales +,"188,793","232,834","263,159","273,046","269,693","291,550","301,938","261,068","249,795","278,454","345,967","437,928","440,061"
1,Expenses +,"164,197","197,980","223,920","234,650","240,104","260,093","277,274","243,081","217,507","253,734","314,151","378,389","379,564"
2,Operating Profit,"24,596","34,853","39,239","38,395","29,589","31,458","24,664","17,987","32,287","24,720","31,816","59,538","60,497"
3,OPM %,13%,15%,15%,14%,11%,11%,8%,7%,13%,9%,9%,14%,14%
4,Other Income +,213,-157,714,"-2,670","1,869","5,933","-26,686",102,"-11,118","2,424","6,664","5,673","6,617"


In [9]:
# Find the section with id="profit-loss"
balance_sheet_section = soup.find('section', id='balance-sheet')

# Extract the table within this section
table = balance_sheet_section.find('table', class_='data-table')

# Extract headers
balance_sheet_headers = [header.text.strip() for header in table.find_all('th')]

# Extract rows
rows = []
for row in table.find_all('tr')[1:]:  # Skip the header row
    cells = [cell.text.strip() for cell in row.find_all('td')]
    rows.append(cells)

# Create a DataFrame
balance_sheet_df = pd.DataFrame(rows, columns=balance_sheet_headers)
balance_sheet_df.head()

,,Mar 2013,Mar 2014,Mar 2015,Mar 2016,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Mar 2024,Sep 2024
0,Equity Capital,638,644,644,679,679,679,679,720,766,766,766,766,736
1,Reserves,"36,999","64,960","55,618","78,273","57,383","94,749","59,500","62,359","54,481","43,795","44,556","84,152","100,326"
2,Borrowings +,"53,716","60,642","73,610","69,360","78,604","88,950","106,175","124,788","142,131","146,449","134,113","107,262","106,549"
3,Other Liabilities +,"76,977","92,180","107,442","114,872","135,914","142,813","139,349","132,313","144,193","138,051","155,239","177,340","183,045"
4,Total Liabilities,"168,330","218,426","237,315","263,184","272,580","327,192","305,703","320,179","341,570","329,061","334,674","369,521","390,656"


In [9]:
# Find the section with id="profit-loss"
cash_flow_section = soup.find('section', id='cash-flow')

# Extract the table within this section
table = cash_flow_section.find('table', class_='data-table')

# Extract headers
cash_flow_section_headers = [header.text.strip() for header in table.find_all('th')]

# Extract rows
rows = []
for row in table.find_all('tr')[1:]:  # Skip the header row
    cells = [cell.text.strip() for cell in row.find_all('td')]
    rows.append(cells)

# Create a DataFrame
cash_flow_df = pd.DataFrame(rows, columns=cash_flow_section_headers)
cash_flow_df


,,Mar 2013,Mar 2014,Mar 2015,Mar 2016,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Mar 2024
0,Cash from Operating Activity +,"22,163","36,151","35,531","37,900","30,199","23,857","18,891","26,633","29,001","14,283","35,388","67,915"
1,Cash from Investing Activity +,"-22,969","-27,991","-36,232","-36,694","-39,571","-25,139","-20,878","-33,115","-25,672","-4,444","-15,417","-22,782"
2,Cash from Financing Activity +,"-1,692","-3,883","5,201","-3,795","6,205","2,012","8,830","3,390","9,904","-3,380","-26,243","-37,006"
3,Net Cash Flow,"-2,499","4,277","4,500","-2,589","-3,167",730,"6,843","-3,092","13,232","6,459","-6,272","8,128"


In [10]:


# Find the section with id="profit-loss"
ratios_section = soup.find('section', id='ratios')

# Extract the table within this section
table = ratios_section.find('table', class_='data-table')

# Extract headers
ratios_section_headers = [header.text.strip() for header in table.find_all('th')]

# Extract rows
rows = []
for row in table.find_all('tr')[1:]:  # Skip the header row
    cells = [cell.text.strip() for cell in row.find_all('td')]
    rows.append(cells)

# Create a DataFrame
ratios_df = pd.DataFrame(rows, columns=ratios_section_headers)
ratios_df


,,Mar 2013,Mar 2014,Mar 2015,Mar 2016,Mar 2017,Mar 2018,Mar 2019,Mar 2020,Mar 2021,Mar 2022,Mar 2023,Mar 2024
0,Debtor Days,21,17,17,18,19,25,23,16,19,16,17,14
1,Inventory Days,64,69,67,73,77,83,73,82,83,71,66,64
2,Days Payable,137,146,131,138,138,151,133,145,175,141,128,126
3,Cash Conversion Cycle,-51,-60,-47,-46,-41,-43,-38,-48,-74,-53,-45,-48
4,Working Capital Days,-57,-41,-44,-47,-50,-50,-53,-68,-48,-32,-24,-24
5,ROCE %,21%,22%,21%,15%,9%,9%,2%,-0%,6%,1%,6%,20%


shareholding

In [11]:
# Find the section with id="profit-loss"
shareholding_section = soup.find('section', id='shareholding')

# Extract the table within this section
table = shareholding_section.find('table', class_='data-table')

# Extract headers
shareholding_section_headers = [header.text.strip() for header in table.find_all('th')]

# Extract rows
rows = []
for row in table.find_all('tr')[1:]:  # Skip the header row
    cells = [cell.text.strip() for cell in row.find_all('td')]
    rows.append(cells)

# Create a DataFrame
shareholding_df = pd.DataFrame(rows, columns=shareholding_section_headers)
shareholding_df


,,Dec 2021,Mar 2022,Jun 2022,Sep 2022,Dec 2022,Mar 2023,Jun 2023,Sep 2023,Dec 2023,Mar 2024,Jun 2024,Sep 2024
0,Promoters +,46.40%,46.40%,46.40%,46.40%,46.39%,46.39%,46.39%,46.38%,46.37%,46.36%,46.36%,42.58%
1,FIIs +,14.57%,14.45%,13.71%,14.13%,13.89%,15.34%,17.72%,18.40%,18.62%,19.20%,18.18%,20.54%
2,DIIs +,13.64%,14.38%,15.17%,14.75%,15.21%,17.69%,17.38%,17.37%,17.25%,16.01%,15.93%,16.08%
3,Government +,0.14%,0.14%,0.14%,0.14%,0.14%,0.14%,0.14%,0.14%,0.14%,0.14%,0.14%,0.29%
4,Public +,25.24%,24.62%,24.57%,24.58%,24.36%,20.41%,18.38%,17.70%,17.60%,18.31%,19.39%,20.49%
5,No. of Shareholders,"32,52,513","37,97,100","40,08,506","39,76,830","39,93,043","38,14,831","35,02,408","37,73,314","41,84,369","46,16,908","50,98,550","57,27,234"


In [12]:

# Find the 'documents' section
documents_section = soup.find('section', id='documents')

# Find all links within the 'documents' section
links = documents_section.find_all('a')

# Extract href and text from each link
link_data = []
for link in links:
    href = link.get('href')
    text = link.text.strip()
    link_data.append([text,href])

# Create a pandas DataFrame
df = pd.DataFrame(link_data, columns=['Text','Link'])


In [13]:
df

,Text,Link
0,All,https://www.bseindia.com/stock-share-price/tat...
1,Announcement under Regulation 30 (LODR)-Press ...,https://www.bseindia.com/stockinfo/AnnPdfOpen....
2,Compliances-Reg. 39 (3) - Details of Loss of C...,https://www.bseindia.com/stockinfo/AnnPdfOpen....
3,Compliances-Reg. 39 (3) - Details of Loss of C...,https://www.bseindia.com/stockinfo/AnnPdfOpen....
4,Announcement under Regulation 30 (LODR)-Analys...,https://www.bseindia.com/stockinfo/AnnPdfOpen....
...,...,...
73,PPT,https://www.bseindia.com/stockinfo/AnnPdfOpen....
74,Transcript,https://www.tatamotors.com/wp-content/uploads/...
75,Transcript,https://www.tatamotors.com/wp-content/uploads/...
76,Transcript,https://www.tatamotors.com/wp-content/uploads/...


In [14]:

# Find the div with class "documents flex-column"
documents_div = soup.find('div', class_='documents flex-column')

data = []

if documents_div:
    # Extract the header
    header = documents_div.find('h3').text.strip()
    print(f"Header: {header}")

    # Find all links within the div
    links = documents_div.find_all('a')

    for link in links:
        href = link.get('href')
        text = link.text.strip()
        
        # Extract the smaller text if it exists
        smaller_div = link.find('div', class_='ink-600 smaller')
        smaller_text = smaller_div.text.strip() if smaller_div else ''
        
        # Add data to the list
        data.append({
            'text': text,
            'url': href,
            'details': smaller_text
        })

    # Create DataFrame
    df = pd.DataFrame(data)
    
    print("\nDataFrame contents:")
    print(df)
    
    # Optionally, save to CSV
    df.to_csv('announcements_data.csv', index=False)
    print("\nData saved to 'announcements_data.csv'")
else:
    print("The specified div was not found.")

Header: Announcements

DataFrame contents:
                                                text   
0                                                All  \
1  Announcement under Regulation 30 (LODR)-Press ...   
2  Compliances-Reg. 39 (3) - Details of Loss of C...   
3  Compliances-Reg. 39 (3) - Details of Loss of C...   
4  Announcement under Regulation 30 (LODR)-Analys...   
5  Compliances-Reg. 39 (3) - Details of Loss of C...   

                                                 url   
0  https://www.bseindia.com/stock-share-price/tat...  \
1  https://www.bseindia.com/stockinfo/AnnPdfOpen....   
2  https://www.bseindia.com/stockinfo/AnnPdfOpen....   
3  https://www.bseindia.com/stockinfo/AnnPdfOpen....   
4  https://www.bseindia.com/stockinfo/AnnPdfOpen....   
5  https://www.bseindia.com/stockinfo/AnnPdfOpen....   

                                             details  
0                                                     
1  2m - Press Release titled ''Tata Motors Monthl...  
2  28 

In [15]:
df

,text,url,details
0,All,https://www.bseindia.com/stock-share-price/tat...,
1,Announcement under Regulation 30 (LODR)-Press ...,https://www.bseindia.com/stockinfo/AnnPdfOpen....,2m - Press Release titled ''Tata Motors Monthl...
2,Compliances-Reg. 39 (3) - Details of Loss of C...,https://www.bseindia.com/stockinfo/AnnPdfOpen....,28 Nov - Intimation of loss of share certifica...
3,Compliances-Reg. 39 (3) - Details of Loss of C...,https://www.bseindia.com/stockinfo/AnnPdfOpen....,
4,Announcement under Regulation 30 (LODR)-Analys...,https://www.bseindia.com/stockinfo/AnnPdfOpen....,25 Nov - Intimation of analyst/institutional i...
5,Compliances-Reg. 39 (3) - Details of Loss of C...,https://www.bseindia.com/stockinfo/AnnPdfOpen....,25 Nov - Intimation of loss of share certifica...


In [16]:

# Find the div with class "documents annual-reports flex-column"
annual_reports_div = soup.find('div', class_='documents annual-reports flex-column')

data = []

if annual_reports_div:
    # Find all links within the div
    links = annual_reports_div.find_all('a')

    for link in links:
        href = link.get('href')
        text = link.text.strip()
        
        # Extract the smaller text if it exists
        smaller_div = link.find('div', class_='ink-600 smaller')
        smaller_text = smaller_div.text.strip() if smaller_div else ''
        
        # Add data to the list
        data.append({
            'Financial Year': text,
            'URL': href,
            'Source': smaller_text
        })

    # Create DataFrame
    df = pd.DataFrame(data)
    
    print("DataFrame contents:")
    # print(df)
    
    # Optionally, save to CSV
    df.to_csv('annual_reports_data.csv', index=False)
    print("\nData saved to 'annual_reports_data.csv'")
else:
    print("The specified div was not found.")

DataFrame contents:

Data saved to 'annual_reports_data.csv'


In [17]:

# Find the div with class "documents credit-ratings flex-column"
credit_ratings_div = soup.find('div', class_='documents credit-ratings flex-column')

data = []

if credit_ratings_div:
    # Find all links within the div
    links = credit_ratings_div.find_all('a')

    for link in links:
        href = link.get('href')
        text = link.find(text=True, recursive=False).strip()
        
        # Extract the smaller text
        smaller_div = link.find('div', class_='ink-600 smaller')
        smaller_text = smaller_div.text.strip() if smaller_div else ''
        
        # Split the smaller text into date and source
        date, source = smaller_text.split(' from ')
        
        # Add data to the list
        data.append({
            'Update Type': text,
            'Date': date.strip(),
            'Source': source.strip(),
            'URL': href
        })

    # Create DataFrame
    df = pd.DataFrame(data)
    
    print("DataFrame contents:")
    # print(df)
    
    # Optionally, save to CSV
    df.to_csv('credit_ratings_data.csv', index=False)
    print("\nData saved to 'credit_ratings_data.csv'")
else:
    print("The specified div was not found.")

DataFrame contents:

Data saved to 'credit_ratings_data.csv'


C:\Users\utamp\AppData\Local\Temp\ipykernel_22432\3346432189.py:12: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  text = link.find(text=True, recursive=False).strip()


In [18]:
df

,Update Type,Date,Source,URL
0,Rating update,7 Oct,crisil,https://www.crisil.com/mnt/winshare/Ratings/Ra...
1,Rating update,5 Jul,care,https://www.careratings.com/upload/CompanyFile...
2,Rating update,13 Jun,icra,https://www.icra.in/Rationale/ShowRationaleRep...
3,Rating update,13 Jun,crisil,https://www.crisil.com/mnt/winshare/Ratings/Ra...
4,Rating update,2 Apr,care,https://www.careratings.com/upload/CompanyFile...
5,Rating update,13 Mar,care,https://www.careratings.com/upload/CompanyFile...


In [19]:

# Find the div with class "documents concalls flex-column"
concalls_div = soup.find('div', class_='documents concalls flex-column')

data = []

if concalls_div:
    # Find all list items within the div
    items = concalls_div.find_all('li', class_='flex flex-gap-8 flex-wrap')

    for item in items:
        # Extract date
        date = item.find('div', class_='ink-600 font-size-15 font-weight-500 nowrap').text.strip()

        # Initialize variables
        transcript = notes = ppt = "N/A"

        # Extract links
        links = item.find_all('a', class_='concall-link')
        for link in links:
            if 'Transcript' in link.text:
                transcript = link['href']
            elif 'PPT' in link.text:
                ppt = link['href']

        # Extract notes (which is a button, not a link)
        notes_button = item.find('button', class_='concall-link', string='Notes')
        if notes_button:
            notes = notes_button['data-url']

        # Add data to the list
        data.append({
            'Date': date,
            'Transcript': transcript,
            'Notes': notes,
            'PPT': ppt
        })

    # Create DataFrame
    df = pd.DataFrame(data)
    
    print("DataFrame contents:")
    # print(df)
    
    # Optionally, save to CSV
    df.to_csv('concalls_data.csv', index=False)
    print("\nData saved to 'concalls_data.csv'")
else:
    print("The specified div was not found.")

DataFrame contents:

Data saved to 'concalls_data.csv'


In [20]:
df

,Date,Transcript,Notes,PPT
0,Nov 2024,https://www.bseindia.com/stockinfo/AnnPdfOpen....,N/A,https://www.bseindia.com/stockinfo/AnnPdfOpen....
1,Aug 2024,https://www.tatamotors.com/wp-content/uploads/...,N/A,https://www.tatamotors.com/wp-content/uploads/...
2,Aug 2024,https://www.bseindia.com/xml-data/corpfiling/A...,N/A,N/A
3,Jun 2024,N/A,N/A,https://www.tatamotors.com/wp-content/uploads/...
4,May 2024,https://www.bseindia.com/xml-data/corpfiling/A...,N/A,https://www.tatamotors.com/wp-content/uploads/...
5,Mar 2024,N/A,N/A,https://aicl-mum-bucket.s3.ap-south-1.amazonaw...
6,Feb 2024,https://www.bseindia.com/xml-data/corpfiling/A...,N/A,https://aicl-mum-bucket.s3.ap-south-1.amazonaw...
7,Feb 2024,N/A,N/A,https://www.tatamotors.com/wp-content/uploads/...
8,Dec 2023,https://www.bseindia.com/stockinfo/AnnPdfOpen....,N/A,https://www.bseindia.com/stockinfo/AnnPdfOpen....
9,Sep 2023,N/A,N/A,https://www.tatamotors.com/wp-content/uploads/...
